In [ ]:
print("Hello World how are You")

In [ ]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from datasets import load_dataset
from haystack import Document
document_store = InMemoryDocumentStore()

In [ ]:
dataset = load_dataset('FreedomIntelligence/medical-o1-reasoning-SFT', 'en', split ='train')


In [76]:
dataset

Dataset({
    features: ['Question', 'Complex_CoT', 'Response'],
    num_rows: 19704
})

In [161]:


dataset


Dataset({
    features: ['Question', 'Complex_CoT', 'Response'],
    num_rows: 19704
})

In [125]:
# limit dataset to top 1000 rows
subset = dataset.select(range(1000))

# build docs list
#docs = [Document(content=doc["Question"], meta=doc["Response"]) for doc in subset]
docs = [
    Document(
        content=doc["Question"], 
        meta={
            "response": doc["Response"],
            "complex_cot": doc["Complex_CoT"]
        }
    )
    for doc in subset
]



In [163]:

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")


doc_embedder.warm_up()

In [164]:
docs_with_embeddings = doc_embedder.run(docs)

Batches: 100%|██████████| 32/32 [00:38<00:00,  1.22s/it]


In [189]:
#docs_with_embeddings

In [ ]:
from haystack.document_stores.in_memory import InMemoryDocumentStore


document_store = InMemoryDocumentStore()  # fresh, empty store


document_store.write_documents(docs_with_embeddings["documents"])

1000

In [137]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

In [138]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store)

In [139]:
retriever

Inputs:
  - query_embedding: list[float]
  - filters: Optional[dict[str, Any]]
  - top_k: Optional[int]
  - scale_score: Optional[bool]
  - return_embedding: Optional[bool]
Outputs:
  - documents: list[Document]

In [170]:
from haystack.components.builders import PromptBuilder

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)

PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


In [171]:
prompt_builder

Inputs:
  - documents: Any
  - question: Any
  - template: Optional[str]
  - template_variables: Optional[dict[str, Any]]
Outputs:
  - prompt: str

In [143]:
import os
from haystack_integrations.components.generators.google_ai import GoogleAIGeminiGenerator
from dotenv import load_dotenv
load_dotenv
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")



In [144]:

gemini = GoogleAIGeminiGenerator(model="gemini-1.5-flash")

In [145]:

res = gemini.run(parts = ["What is this document about?"])

In [103]:
res

{'replies': ['Please provide me with the document. I need the text of the document to tell you what it is about.\n']}

In [146]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()

In [167]:
prompt_builder

prompt_builder
Inputs:
  - documents: Any
  - question: Any
  - template: Optional[str]
  - template_variables: Optional[dict[str, Any]]
Outputs:
  - prompt: str

In [147]:
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", gemini)

In [148]:
# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: GoogleAIGeminiGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (list[float])
  - retriever.documents -> prompt_builder.documents (list[Document])
  - prompt_builder.prompt -> llm.parts (str)

In [151]:
question = "hat is the most likely diagnosis for a 2-year-old 70 kg child?"

In [ ]:
#response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

response = basic_rag_pipeline.run(
    {"text_embedder": {"text": question},"prompt_builder": {"question": question}},
    include_outputs_from={"retriever", "prompt_builder"})

Batches: 100%|██████████| 1/1 [00:00<00:00, 38.46it/s]


In [187]:
print(response.keys())
print(response)


dict_keys(['retriever', 'prompt_builder', 'llm'])
{'retriever': {'documents': [Document(id=645044a230641633ed49bad016c572decde07df578712f8e6fe0e1bd3fa99912, content: 'What is the most likely diagnosis for a 2-year-old 70 kg child who presents with limitation of abduc...', meta: {'response': 'Based on the described symptoms and the unusual weight for a 2-year-old child, the most likely diagnosis is Slipped Capital Femoral Epiphysis (SCFE). Even though SCFE typically occurs in older children, mainly adolescents, the combination of excessive weight, limited hip abduction and internal rotation, tenderness in the hip area, and the characteristic limb movement (abduction upon hip flexion) strongly points towards SCFE as the most plausible diagnosis in this scenario.', 'complex_cot': "Okay, so we're dealing with a 2-year-old child here. But wait, this kid is listed as 70 kg – that's a lot for a toddler! It might be a typo, or maybe there's something else going on. Anyway, let's focus on the s

In [190]:
#response

In [191]:
# # Example query
# query = "What is generative AI?"

# # Run the pipeline
# result = basic_rag_pipeline.run({
#     "text_embedder": {"text": query}, "prompt_builder": {"question": question}
# })

# # 🔎 Check retrieved documents
# retrieved_docs = response["retriever"]["documents"]
# # print("\n=== Retrieved Documents ===")
# # for i, doc in enumerate(retrieved_docs, 1):
# #     print(f"\nDocument {i}:")
# #     print("Content:", doc.content)
# #     print("Meta:", doc.meta)

# # 🔎 Check the built prompt
# prompt_text = response["prompt_builder"]["prompt"]
# print("\n=== Built Prompt ===")
# print(prompt_text)

# # ✅ Final LLM answer
# print("\n=== LLM Answer ===")
# print(response["llm"]["replies"][0])


In [154]:
response['llm']['replies'][0]

"The most likely diagnosis for a 2-year-old 70 kg child presenting with limitation of abduction and internal rotation, tenderness in Scarpa's triangle, and abduction of the limb upon flexing the hip is **a septic hip (septic arthritis of the hip)**.  The extreme weight (70 kg is significantly above the normal weight for a 2-year-old) is not directly related to the diagnosis but highlights the unusual presentation.  The other symptoms strongly suggest a septic joint.\n"

In [158]:
question="What is the most likely diagnosis for a 2-year-old 70 kg child?"

In [159]:
response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

Batches: 100%|██████████| 1/1 [00:00<00:00, 22.22it/s]


In [160]:
response["llm"]["replies"][0]

"The provided text gives symptoms, but doesn't state a diagnosis.  A 2-year-old weighing 70 kg is extremely obese, and this extreme weight itself would be a major concern requiring investigation.  The symptoms of limitation of abduction and internal rotation, tenderness in Scarpa's triangle, and abduction of the limb upon flexing the hip suggest a possible hip problem, perhaps septic arthritis or another infection. However, without further information, a precise diagnosis cannot be made.  The extreme weight should be a primary consideration in any differential diagnosis.\n"